In [1]:
!pip install experta

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for frozendict, since package 'wheel' is not installed.
  Running setup.py install for frozendict: started
  Running setup.py install for frozendict: finished with status 'done'


You should consider upgrading via the 'C:\Users\Usuario\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
# Para compatibilidad con versiones anteriores, siempre utilizarlo.
import collections.abc
if not hasattr(collections, 'Mapping'):
    collections.Mapping = collections.abc.Mapping

In [3]:
from experta import *  # Importa la libreria experta para crear el sistema experto

# Definicion de hechos utilizados en el sistema
class Symptom(Fact):  # Representa un sintoma observado en el vehiculo
    pass

class CarState(Fact):  # Estado general del motor u otra parte del vehiculo
    pass

class Diagnosis(Fact):  # Diagnostico generado por el sistema
    pass

class RepairAction(Fact):  # Representa una accion de reparacion necesaria
    pass

class VehicleStatus(Fact):  # Estado general del proceso de reparacion
    pass

# Clase principal que contiene las reglas del sistema experto
class VehicleDiagnosis(KnowledgeEngine):

    # Regla: humo blanco y luz de aceite indican posible dano grave al motor
    @Rule(AND(Symptom(tipo='humo_blanco'), Symptom(tipo='luz_aceite')), salience=100)
    def motor_grave_dano(self):
        resultado = 'Posible_junta_de_cabeza_mala,_motor_en_riesgo'
        print(resultado)
        self.declare(Diagnosis(resultado=resultado))
        self.declare(RepairAction(tipo='reparar_motor'))

    # Regla: ruido metalico indica posible problema en los frenos
    @Rule(Symptom(tipo='ruido_metalico'), salience=90)
    def frenos_problema(self):
        resultado = 'Revisar_sistema_de_frenos'
        print(resultado)
        self.declare(Diagnosis(resultado=resultado))
        self.declare(RepairAction(tipo='reparar_frenos'))

    # Complete la regla: fuga de liquido + motor caliente = posible perdida de refrigerante
    @Rule(AND(Symptom(tipo='fuga_liquido'), CarState(estado='motor_caliente')), salience=80)
    def perdida_refrigerante(self):
        resultado = 'Perdida_de_refrigerante,_posible_sobrecalentamiento'
        print(resultado)
        self.declare(Diagnosis(resultado=resultado))
        self.declare(RepairAction(tipo='rellenar_refrigerante'))

    # Regla: si no hay sintomas clave, sugerir revision general
    @Rule(NOT(Symptom(tipo='humo_blanco')), NOT(Symptom(tipo='luz_aceite')), NOT(Symptom(tipo='ruido_metalico')), salience=10)
    def revision_general(self):
        resultado = 'Revision_general_recomendada'
        print(resultado)
        self.declare(Diagnosis(resultado=resultado))

    # Regla de reparacion: eliminar sintoma 'ruido_metalico'
    @Rule(RepairAction(tipo='reparar_frenos'), Symptom(tipo='ruido_metalico'), salience=70)
    def ejecutar_reparacion_frenos(self):
        print("Reparando sistema de frenos - Eliminando sintoma 'ruido_metalico'")
        for fact in self.facts:
            if isinstance(fact, Symptom) and self.facts[fact]['tipo'] == 'ruido_metalico':
                self.retract(fact)
                break
        print("Sintoma eliminado, vehiculo reparado")
        self.declare(VehicleStatus(estado="verificar_reparacion"))

    # Regla de reparacion: eliminar sintoma 'humo_blanco'
    @Rule(RepairAction(tipo='reparar_motor'), Symptom(tipo='humo_blanco'), salience=60)
    def ejecutar_reparacion_motor_humo(self):
        print("Reparando problema del motor - Eliminando sintoma 'humo_blanco'")
        for fact in self.facts:
            if isinstance(fact, Symptom) and self.facts[fact]['tipo'] == 'humo_blanco':
                self.retract(fact)
                break
        self.declare(VehicleStatus(estado='en_reparacion'))
        print("Sintoma de humo eliminado, motor en reparacion")
        self.declare(VehicleStatus(estado="verificar_reparacion"))

    # Regla de reparacion: eliminar sintoma 'luz_aceite'
    @Rule(RepairAction(tipo='reparar_motor'), Symptom(tipo='luz_aceite'), salience=50)
    def ejecutar_reparacion_motor_aceite(self):
        print("Reparando problema del motor - Eliminando sintoma 'luz_aceite'")
        for fact in self.facts:
            if isinstance(fact, Symptom) and self.facts[fact]['tipo'] == 'luz_aceite':
                self.retract(fact)
                break
        print("Sintoma de luz de aceite eliminado")
        self.declare(VehicleStatus(estado="verificar_reparacion"))

    # Verifica si ya no quedan sintomas: vehiculo esta listo
    @Rule(VehicleStatus(estado="verificar_reparacion"), salience=20)
    def vehiculo_reparado(self):
        tiene_sintomas = False
        for fact in self.facts:
            if isinstance(fact, Symptom):
                tiene_sintomas = True
                break

        if not tiene_sintomas:
            print("Todos los sintomas han sido resueltos. Vehiculo listo.")

# Ejecucion del sistema experto
if __name__ == "__main__":
    engine = VehicleDiagnosis()
    engine.reset()  # Reinicia el motor

    print("Primera ejecucion - Diagnostico inicial")
    # Complete las declaraciones
    engine.declare(Symptom(tipo='humo_blanco'))
    engine.declare(Symptom(tipo='luz_aceite'))
    engine.declare(Symptom(tipo='ruido_metalico'))
    engine.declare(CarState(estado='motor_caliente'))
    engine.declare(Symptom(tipo='fuga_liquido'))

    engine.run()  # Ejecuta las reglas

Primera ejecucion - Diagnostico inicial
Posible_junta_de_cabeza_mala,_motor_en_riesgo
Revisar_sistema_de_frenos
Perdida_de_refrigerante,_posible_sobrecalentamiento
Reparando sistema de frenos - Eliminando sintoma 'ruido_metalico'
Sintoma eliminado, vehiculo reparado
Reparando problema del motor - Eliminando sintoma 'humo_blanco'
Sintoma de humo eliminado, motor en reparacion
Reparando problema del motor - Eliminando sintoma 'luz_aceite'
Sintoma de luz de aceite eliminado
Todos los sintomas han sido resueltos. Vehiculo listo.
